In [1]:
import numpy as np
import qiskit as qk
from qiskit import Aer
from U_unitary import *
from W_unitary import *
from qiskit.visualization import plot_histogram

In [2]:
def W_original(circuit,theta,n,d):
    #Create circuit
    circuit = W(circuit,theta,n,d)
    circuit.barrier()
    circuit.z(range(n))
    circuit.measure(range(n),range(n))

    return circuit

def expectation_groundstate(circuit,shots,n):
    simulator = Aer.get_backend('aer_simulator')
    result = simulator.run(circuit,shots=shots).result()
    counts = result.get_counts()
    total_counts = sum(counts.values())
    string = '0'*n
    if string in counts:
        expectation_notnorm = counts[string]
    else:
        expectation_notnorm = 0
    expectation_groundstate = expectation_notnorm/total_counts
    return expectation_groundstate

In [3]:
def apply_W_top_before_split(qc,theta, n, n_part, l=1):
    # Corresponds to U_top
    Theta_matrix = np.reshape(theta, (n,l,3))[0:2,:,:]
    for j in range(l):
        for i in range(n_part):
            theta_gate(qc,Theta_matrix[i,j,:],i)
    qc.cz(0,1)        
    return qc

def apply_W_top_after_split(qc):
    # Corresponds to V_top
    return qc

def apply_W_bottom_before_split(qc,theta, n, n_part, l=1):
    # Corresponds to U_bot
    Theta_matrix = np.reshape(theta, (n,l,3))[2:4,:,:]
    for j in range(l):
        for i in range(n_part):
            theta_gate(qc,Theta_matrix[i,j,:],i)
    return qc

def apply_W_bottom_after_split(qc):
    # Corresponds to V_bot
    qc.cz(0,1)   
    return qc

def apply_u_gate(qc,dag,index_u):
    phi = (-1)**dag*(np.pi/2)
    qc.p(phi ,index_u)
    return qc

def top_circuit(theta, n, n_part,dag):
    circuit = qk.QuantumCircuit(n_part,n_part)

    circuit = apply_W_top_before_split(circuit,theta, n, n_part)
    circuit = apply_u_gate(circuit,dag,1)
    # circuit = apply_W_top_after_split(circuit)

    circuit.barrier()
    circuit.z(range(n_part))
    circuit.measure(range(n_part),range(n_part)) 
    return circuit

def bottom_circuit(theta, n, n_part,dag):
    circuit = qk.QuantumCircuit(n_part,n_part)

    circuit = apply_W_bottom_before_split(circuit,theta, n, n_part)
    circuit = apply_u_gate(circuit,dag,0)
    circuit = apply_W_bottom_after_split(circuit)

    circuit.barrier()
    circuit.z(range(n_part))
    circuit.measure(range(n_part),range(n_part)) 
    return circuit

In [ ]:
def theta_gate_conj(qc,theta,i):
    assert len(theta) == 3, "theta incorrect size"
    rx_matrix_conj = [[np.cos(theta[0]/2), 1j*np.sin(theta[0]/2)],[1j*np.sin(theta[0]/2), np.cos(theta[0]/2)]]
    rz_matrix_conj = [[np.exp(1j*theta[2]/2),0,[0, np.exp(-1j*theta[2]/2)]]]
    qc.append(qk.Operator(rx_matrix_conj),[i])
    qc.ry(theta[1],i)
    qc.append(qk.Operator(rz_matrix_conj),[i])
    return 0


def apply_W_top_before_split_conj(qc):
    # Corresponds to V_top_dagger
      
    return qc

def apply_W_top_after_split_conj(qc,theta, n, n_part, l=1):
    # Corresponds to U_top_dagger
    Theta_matrix = np.reshape(theta, (n,l,3))[0:2,:,:]
    for j in range(l):
        for i in range(n_part):
            theta_gate_conj(qc,Theta_matrix[i,j,:],i)
    qc.cz(0,1)  
    return qc

def apply_W_bottom_before_split_conj(qc,theta, n, n_part, l=1):
    # Corresponds to V_bot_dagger
    Theta_matrix = np.reshape(theta, (n,l,3))[2:4,:,:]
    for j in range(l):
        for i in range(n_part):
            theta_gate(qc,Theta_matrix[i,j,:],i)
    return qc

def apply_W_bottom_after_split_conj(qc):
    # Corresponds to U_bot_dagger
    qc.cz(0,1)   
    return qc

def top_circuit_conj(theta, n, n_part,dag):
    circuit = qk.QuantumCircuit(n_part,n_part)

    circuit = apply_W_top_before_split_conj(circuit,theta, n, n_part)
    circuit = apply_u_gate(circuit,1-dag,1)
    circuit = apply_W_top_after_conj(circuit)

    circuit.barrier()
    circuit.z(range(n_part))
    circuit.measure(range(n_part),range(n_part)) 
    return circuit

def bottom_circuit_conj(theta, n, n_part,dag):
    circuit = qk.QuantumCircuit(n_part,n_part)

    circuit = apply_W_bottom_before_split_conj(circuit,theta, n, n_part)
    circuit = apply_u_gate(circuit,1-dag,0)
    circuit = apply_W_bottom_after_split_conj(circuit)

    circuit.barrier()
    circuit.z(range(n_part))
    circuit.measure(range(n_part),range(n_part)) 
    return circuit

def apply_u_gate(qc,dag,index_u):
    phi = (-1)**dag*(np.pi/2)
    qc.p(phi ,index_u)
    return qc






In [7]:
n = 4
n_part = 2
d = 1
theta = 2*np.pi*np.random.random(n*d*3)


circuit_test = qk.QuantumCircuit(n,n)
shots = 1000000

W_original_test = W_original(circuit_test,theta,n,d)
expectation_original = expectation_groundstate(W_original_test,shots,n)
print(expectation_original)


top_test_1 = top_circuit(theta, n, n_part,0)
top_test_2 = top_circuit(theta, n, n_part,1)
bottom_test_1 = bottom_circuit(theta, n, n_part,0)
bottom_test_2 = bottom_circuit(theta, n, n_part,1)

expectation_top_1 = expectation_groundstate(top_test_1,shots,n_part)
expectation_top_2 = expectation_groundstate(top_test_2,shots,n_part)
expectation_bottom_1 = expectation_groundstate(bottom_test_1,shots,n_part)
expectation_bottom_2 = expectation_groundstate(bottom_test_2,shots,n_part)

expectation_partitioned = 1/2*(expectation_top_1*expectation_bottom_1 + expectation_top_2*expectation_bottom_2)
print(expectation_partitioned)

0.064734
0.064609817396
